In [1]:
import pandas as pd
import data_reader as dr
import feature_engineering_SA as fe_s
import numpy as np
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [2]:
import os
os.chdir("/Users/sakshamarora/Documents/6748 Practicum/project/GT_MSA_LBA")

## Current Season Data

In [3]:
league = "NCAAW1+"
season = "22-23"
past_season = "21-22"
raw_data_path = "data/raw_data/"
dataframes_cs = dr.read_data(raw_data_path,league,season)

INFO:root:['NCAAW1+ 22-23 shotsdf.pickle', 'NCAAW1+ 22-23 teamsdf.pickle', 'NCAAW1+ 22-23 segment player block and usage data.pickle', 'NCAAW1+ 22-23 segment data.pickle', 'NCAAW1+ 22-23 gamesdf.pickle', 'NCAAW1+ 22-23 playerdictionary.pickle']
INFO:root:reading files from data/raw_data/NCAAW1+/S-22-23
INFO:root:files present: ['NCAAW1+ 22-23 shotsdf.pickle', 'NCAAW1+ 22-23 teamsdf.pickle', 'NCAAW1+ 22-23 segment player block and usage data.pickle', 'NCAAW1+ 22-23 segment data.pickle', 'NCAAW1+ 22-23 gamesdf.pickle', 'NCAAW1+ 22-23 playerdictionary.pickle']


In [4]:
errors = dr.validate_data_types(dataframes_cs)


330    None
456    None
Name: hometeam, dtype: object
Name: player_id, dtype: object
1    618ade0180945020656b0939
2    618ade0180945020656b0969
Name: team_id, dtype: object
1    618ade0180945020656b093b
2    618ade0180945020656b096b
Name: season_id, dtype: object
1    64e4d36650f218f22c415b72
2    64e4d36650f218f22c415b6b
Name: conferenceid, dtype: object


### Manually Fix data errors found, drop if minimal missing data

In [5]:
# TODO: USE IF NECESSARY TO CLEAN DATA
# dataframes['segments_df'] = dataframes['segments_df'].loc[~(dataframes['segments_df'].index.isin(errors['segments_df__players_team2'].index) | (dataframes['segments_df'].index.isin(errors['segments_df__players_team1'].index)))].reset_index(drop=True)
# dataframes['segments_players_usage_blocking_df'] = dataframes['segments_players_usage_blocking_df'].loc[~dataframes['segments_players_usage_blocking_df'].index.isin(errors['segments_players_usage_blocking_df__player_id'].index)].reset_index(drop=True)
# dataframes['shots_df'] = dataframes['shots_df'].loc[~dataframes['shots_df'].index.isin(errors['shots_df__player_id'].index)].reset_index(drop=True)


In [6]:
dataframes_cs['shots_df'] = dr.clean_shots_data(dataframes_cs['shots_df'])
dataframes_cs['games_df'].date = pd.to_datetime(dataframes_cs['games_df'].date)

/var/folders/pb/ltqtnlc15yxbdykyrh7hhqg80000gn/T/ipykernel_23783/2808550072.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframes_cs['games_df'].date = pd.to_datetime(dataframes_cs['games_df'].date)


## Past Season Data

In [7]:
dataframes_ps = dr.read_data(raw_data_path,league,past_season)

INFO:root:['NCAAW1+ 21-22 shotsdf.pickle', 'NCAAW1+ 21-22 teamsdf.pickle', 'NCAAW1+ 21-22 segment player block and usage data.pickle', 'NCAAW1+ 21-22 gamesdf.pickle', 'NCAAW1+ 21-22 playerdictionary.pickle', 'NCAAW1+ 21-22 segment data.pickle']
INFO:root:reading files from data/raw_data/NCAAW1+/S-21-22
INFO:root:files present: ['NCAAW1+ 21-22 shotsdf.pickle', 'NCAAW1+ 21-22 teamsdf.pickle', 'NCAAW1+ 21-22 segment player block and usage data.pickle', 'NCAAW1+ 21-22 gamesdf.pickle', 'NCAAW1+ 21-22 playerdictionary.pickle', 'NCAAW1+ 21-22 segment data.pickle']


In [8]:
errors = dr.validate_data_types(dataframes_ps)

393    None
394    None
Name: hometeam, dtype: object
Name: player_id, dtype: object
1    618ade0180945020656b0935
2    618ade0180945020656b0939
Name: team_id, dtype: object
1    618ade0180945020656b0936
2    618ade0180945020656b093a
Name: season_id, dtype: object
1    64e4d36650f218f22c415b7e
2    64e4d36650f218f22c415b72
Name: conferenceid, dtype: object


In [9]:
# # TODO: MANUAL USE IF NECESSARY TO CLEAN DATA
# dataframes_ps['segments_df'] = dataframes_ps['segments_df'].loc[~(dataframes_ps['segments_df'].index.isin(errors['segments_df__players_team2'].index) | (dataframes_ps['segments_df'].index.isin(errors['segments_df__players_team1'].index)))].reset_index(drop=True)
# dataframes_ps['segments_players_usage_blocking_df'] = dataframes_ps['segments_players_usage_blocking_df'].loc[~dataframes_ps['segments_players_usage_blocking_df'].index.isin(errors['segments_players_usage_blocking_df__player_id'].index)].reset_index(drop=True)
# #The 'Pullup' and 'Hook' columns are redundant and thus will be removed.
# dataframes_ps['shots_df'] = dataframes_ps['shots_df'].drop(columns=['Pullup','Hook'])
# dataframes_ps['shots_df']['Fastbreak'] = np.where(dataframes_ps['shots_df']['Fast Break']==True, True, dataframes_ps['shots_df']['Fastbreak'])
# dataframes_ps['shots_df'] = dataframes_ps['shots_df'].drop(columns=['Fast Break'])
# dataframes_ps['shots_df'] = dataframes_ps['shots_df'].rename(columns={'Pull Up':'Pullup'})
# # dataframes['shots_df'] = dataframes['shots_df'].loc[~dataframes['shots_df'].index.isin(errors['shots_df__player_id'].index)].reset_index(drop=True)

In [10]:
dataframes_ps['shots_df'] = dr.clean_shots_data(dataframes_ps['shots_df'])


In [11]:
df_shots = dataframes_cs['shots_df']
df_games = dataframes_cs['games_df']
df_players = dataframes_cs['players_df']
df_team = dataframes_cs['teams_df']

# Feature Creation

## Feature Part 1

In [12]:
# Drop irrelevant features
features_df = df_shots[['game_id', 'team_id', 'season_id', 'player_id', 'shot_id','segment_id','Distance', 'Angle', 'Zone', 'x_coordinate', 'y_coordinate', 'Period','Time_Seconds','Time','score','score_opp','Stat']].copy()

# For shot/play type, we'll take multiple columns
# TODO: ADD BOOLEAN FEATURES HERE
shot_play_type_columns = ['Jumpshot', 'Assisted', 'Red Zone', 'Floater', 'Second Chance', 'Layup', 'Paint', 'Drive', 'Blocked', 'ATO', 'Off Turnover', 'Off Steal', 'Hook Shot', 'Fastbreak', 'Pullup', 'And1', 'Turnaround', 'Dunk', 'Step Back']
features_df[shot_play_type_columns] = df_shots[shot_play_type_columns]

In [13]:
merged_df = pd.merge(features_df, df_games, on='game_id', how='left')

# Create 'Location' feature based on 'team_id' and 'hometeam'
def determine_location(row):
    if pd.isna(row['hometeam']):
        return 'Neutral'
    elif row['team_id'] == row['hometeam']:
        return 'Home'
    else:
        return 'Away'

merged_df['Location'] = merged_df.apply(determine_location, axis=1)

### Adding win-loss ratio

In [14]:
periods = [10, 5, 3, 1]

def compute_win_loss_ratios_for_period(df_games, team_id, period):
    """Compute win-loss ratios for a team for the given period."""
    # Filter the team's games
    team_games = df_games[(df_games['team1'] == team_id) | (df_games['team2'] == team_id)]
    
    # Sort by date
    team_games = team_games.sort_values(by='date')
    
    # Compute win flags (1 if won, 0 otherwise)
    team_games['won'] = (team_games['winning_team'] == team_id).astype(int)
    
    # Calculate rolling win sum and count, excluding the current game
    team_games[f'wins_last_{period}'] = team_games['won'].shift(1).rolling(window=period, min_periods=1, closed='left').sum()
    team_games[f'games_last_{period}'] = team_games['won'].shift(1).rolling(window=period, min_periods=1, closed='left').count()
    
    # Calculate ratio
    team_games[f'w_l_ratio_last_{period}'] = team_games[f'wins_last_{period}'] / team_games[f'games_last_{period}']
    
    return team_games[['date', f'w_l_ratio_last_{period}']].set_index('date')

# Calculate win-loss ratios for all teams and periods in advance
team_ids = df_games['team1'].unique()
ratios_cache = {}
for team_id in team_ids:
    for period in periods:
        ratios_cache[(team_id, period)] = compute_win_loss_ratios_for_period(df_games, team_id, period)

def get_opponent(row):
    return row['team2'] if row['team_id'] == row['team1'] else row['team1']

def get_ratio_from_cache_vectorized(dates, team_ids, period):
    """Vectorized function to fetch pre-calculated ratio from the cache."""
    results = []
    for date, team_id in zip(dates, team_ids):
        ratios = ratios_cache.get((team_id, period))
        if ratios is not None:
            results.append(ratios.loc[date, f'w_l_ratio_last_{period}'] if date in ratios.index else float('nan'))
        else:
            results.append(float('nan'))
    return results

for period in tqdm(periods, desc='Updating DataFrame'):
    # Team stats
    merged_df[f'w_l_ratio_last_{period}'] = get_ratio_from_cache_vectorized(merged_df['date'], merged_df['team_id'], period)
    # Opposition stats
    merged_df[f'opp_w_l_ratio_last_{period}'] = get_ratio_from_cache_vectorized(merged_df['date'], merged_df.apply(get_opponent, axis=1), period)

Updating DataFrame: 100%|██████████| 4/4 [00:59<00:00, 14.80s/it]


### Adding player attributes

In [15]:
merged_df = pd.merge(merged_df, df_players, left_on='player_id', right_on='player', how='left')
merged_df.drop(columns=['player', 'Genius_position','seasonid'], inplace=True)


### Adding previous season win-loss ratio

In [16]:
df_games_prev_season = dataframes_ps["games_df"]

### Adding previous season win_loss ratio

In [17]:
def compute_prev_season_win_loss_ratio(df_games_prev_season):
    team_wins = df_games_prev_season.groupby('winning_team').size()
    team_games = df_games_prev_season.groupby(['team1']).size() + df_games_prev_season.groupby(['team2']).size()

    # Win-loss ratio
    win_loss_ratio = team_wins / team_games
    return win_loss_ratio

win_loss_ratio_prev_season = compute_prev_season_win_loss_ratio(df_games_prev_season)

def get_team_wl_ratio(team_id, win_loss_ratio):
    return win_loss_ratio.get(team_id, float('nan'))

merged_df['team_wl_ratio_prev_season'] = merged_df['team_id'].apply(lambda x: get_team_wl_ratio(x, win_loss_ratio_prev_season))
merged_df['opp_team_wl_ratio_prev_season'] = merged_df.apply(lambda row: get_team_wl_ratio(get_opponent(row), win_loss_ratio_prev_season), axis=1)

In [18]:
import pandas as pd
import numpy as np
import os
import pickle
from bson.objectid import ObjectId
from collections import defaultdict
import timeit
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import datetime

import data_reader as dr
import feature_engineering_SA as fe_s

## Features Part 2

### Creating Time Remaining on Game Clock, Score Difference, Clutch Time

In [19]:
shot_id_time_features = fe_s.get_time_score_features(dataframes_cs['shots_df'])

In [20]:
feature_list_1_shot_level = shot_id_time_features
feature_list_1_shot_level

,shot_id,Time_Seconds,score_difference,clutch_time
0,636a63d48efc1cede33f6085,496,-3,0
1,636a63d48efc1cede33f6086,471,-6,0
2,636a63d48efc1cede33f6088,449,-6,0
3,636a63d48efc1cede33f608a,419,-7,0
4,636a63d48efc1cede33f6095,391,-5,0
...,...,...,...,...
664649,645d57b639354d6a0fda3380,64,-11,0
664650,645d57b639354d6a0fda3382,39,-8,0
664651,645d57b639354d6a0fda3386,17,-10,0
664652,645d57b639354d6a0fda338a,9,-11,0


### Creating Avg./max blocking rates of opponent team on court (segment)

### Game Player Level Blocking and Usage Stats 
#### 1. Get All players in each segment to make at same level as blocking stats df

In [21]:
segments_df_long = fe_s.create_player_level_segment_df(dataframes_cs['segments_df'])

In [22]:
segment_master_df, segment_master_df_game_player = fe_s.get_player_game_stats(segments_df_long,dataframes_cs['segments_players_usage_blocking_df'])

In [23]:
team_game_player_stats_df = fe_s.get_all_team_player_game_stats(segment_master_df,segment_master_df_game_player,dataframes_cs['games_df'])


As we have stats for each player for a game, now we can join this info at segment level and calculate averages for all players of opponent team

In [24]:
fe_s.get_segment_team_level_blockers(team_game_player_stats_df,segments_df_long,choose_threshold=True)

(0.0, 0.03125, 0.05555555555555555, 0.08333333333333333)
(0.02247191011235955, 0.027777777777777776, 0.045454545454545456, 0.06451612903225806)
0.021739130434782608 0.02631578947368421 0.041666666666666664 0.058823529411764705
0.021551724137931036 0.02631578947368421 0.04176904176904177 0.058823529411764705


In [25]:
segment_team_level_blockers = fe_s.get_segment_team_level_blockers(team_game_player_stats_df,segments_df_long) #BLOCKING_THRESHOLD=0.0306
feature_list_2_game_segment_team_level = segment_team_level_blockers

In [26]:
feature_list_3_game_segment_player_level = fe_s.get_segment_usage_ratios(segments_df_long,team_game_player_stats_df)

#### Feature: Opposition Player Characteristics

In [27]:
feature_list_4_game_segment_team_level = fe_s.get_segment_positions(segments_df_long,dataframes_cs['players_df'])

Previous Season Feature creation

In [28]:
segments_df_long = fe_s.create_player_level_segment_df(dataframes_ps['segments_df'])

In [29]:
segment_master_df, segment_master_df_game_player = fe_s.get_player_game_stats(segments_df_long,dataframes_ps['segments_players_usage_blocking_df'])

In [30]:
player_past_season_stats = fe_s.get_all_team_player_stats_last_season(segment_master_df_game_player)

In [31]:
feature_list_5_player_level = player_past_season_stats

In [32]:
altered_shots_df = fe_s.get_corrected_score_feature(dataframes_cs['shots_df'])

Merging all features

In [33]:
shots = fe_s.get_all_features_at_shot_level(altered_shots_df,dataframes_cs['segments_df'],feature_list_1_shot_level,feature_list_2_game_segment_team_level,feature_list_3_game_segment_player_level,feature_list_4_game_segment_team_level,feature_list_5_player_level)

In [34]:
subset_cols = ['shot_id',
       'score_pre_shot','score_pre_shot_diff', 'clutch_time',
       'num_blockers_on_team', 'last_1_game_usage_ratio',
       'last_3_game_usage_ratio', 'last_5_game_usage_ratio',
       'last_10_game_usage_ratio','current_season_usage_ratio','past_season_usage_ratio', 'Big', 'Mid', 'Small']

In [35]:
features_part_2 = shots[subset_cols]

In [36]:
features_part_2

,shot_id,score_pre_shot,score_pre_shot_diff,clutch_time,num_blockers_on_team,last_1_game_usage_ratio,last_3_game_usage_ratio,last_5_game_usage_ratio,last_10_game_usage_ratio,current_season_usage_ratio,past_season_usage_ratio,Big,Mid,Small
0,636a63d48efc1cede33f6085,0,-3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1
1,636a63d48efc1cede33f6086,0,-6,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1
2,636a63d48efc1cede33f6088,0,-6,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1
3,636a63d48efc1cede33f608a,0,-7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1
4,636a63d48efc1cede33f6095,0,-7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664649,645d57b639354d6a0fda3380,73,-11,0,1,0.169492,0.173679,0.170033,0.190997,0.197837,0.228142,2,2,1
664650,645d57b639354d6a0fda3382,73,-11,0,1,0.212766,0.221935,0.226966,0.216381,0.198397,0.275670,2,2,1
664651,645d57b639354d6a0fda3386,76,-10,0,1,0.345882,0.324706,0.335132,0.320195,0.331507,0.334756,2,3,0
664652,645d57b639354d6a0fda338a,76,-11,0,0,0.169492,0.173679,0.170033,0.190997,0.197837,0.228142,1,4,0


## Features Part 3

### Past Season

In [37]:
#Import neccessary pickle files
shots = dataframes_ps['shots_df'].copy()
player = dataframes_ps['players_df'].copy()
segment = dataframes_ps['segments_df'].copy()

In [38]:
segment = segment[['game_id','players_team1','players_team2']]
segment["all_players"] = segment["players_team1"]+segment["players_team2"]
segments_df_long = segment.explode(["all_players"])
segments_df_long = segments_df_long.drop(columns=['players_team1','players_team2'])
segments_df_long = segments_df_long.drop_duplicates()

In [39]:
segments_df_long['played'] = 1
segments_df_long = segments_df_long.drop(columns=['game_id']).groupby(['all_players']).sum().reset_index()

Create a dataframe with all possible combinations of player_id, season_id, team_id

In [40]:
#merge the players with the shots dataframe to add the team_id column 
#that is neccessary for merging with the games dataframe
player_team = pd.merge(player[['player','seasonid']].rename(columns={'seasonid':'season_id','player':'player_id'}),
    shots[['team_id','season_id']].drop_duplicates(), how='left', on='season_id')

In [41]:
player_team = pd.merge(player_team, segments_df_long.rename(columns={'all_players':'player_id'}), on=['player_id'], how='left')
player_team.played = player_team.played.fillna(0).astype(int)

In [42]:
#Split the Stat column into 2s/3s and Makes/Misses
shots_df = shots.copy()
shots_df.insert(13, 'Type', shots_df.Stat.str[:-4])
shots_df.insert(14, 'Result', shots_df.Stat.str[-4:])
shots_df.Result = np.where(shots_df.Result=='Make', 1, 0)

Feature creation for 2 pointers

In [43]:
#Filter on 2 point shots
Twos_df = shots_df[shots_df.Type=='Two']
two_pointers_df = Twos_df.groupby(['player_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'2PM', 'count':'2PA'})

In [44]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
two_pointers_df = pd.merge(player_team[['player_id','played']], two_pointers_df, how= 'left', on='player_id').fillna(0)

In [45]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Dunk', 'Layup', 'Floater', 'Hook Shot', 'Jumpshot', 'Pullup', 'Turnaround', 'Step Back']:
    temp = Twos_df[Twos_df[stat]==True]
    temp_df = temp.groupby(['player_id']).Result.agg(['sum','count']).reset_index().rename(
        columns={'sum': stat+'s made', 'count': stat+'s attempted'})
    two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)

In [46]:
#Calculated percentage of made 2p jumpshots that were assisted
temp_df = Twos_df[(Twos_df['Jumpshot']==True)&(Twos_df['Result']==1)].groupby(
    ['player_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': 'Jumpshots assisted on'})

In [47]:
two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)

In [48]:
#2P% per zone for each player
Zones2 = Twos_df.groupby(['player_id','Zone']).Result.agg(['sum','count']).reset_index()

In [49]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones2.pivot(index=['player_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])#remove leading space

In [50]:
two_pointers_df = pd.merge(two_pointers_df, pivot_zones, how= 'left', on=['player_id']).fillna(0)

In [51]:
two_pointers_df.insert(4, '2P%', two_pointers_df['2PM']/two_pointers_df['2PA'])
two_pointers_df.insert(7, 'Dunk FG%', two_pointers_df['Dunks made']/two_pointers_df['Dunks attempted'])
two_pointers_df.insert(10, 'Layup FG%', two_pointers_df['Layups made']/two_pointers_df['Layups attempted'])
two_pointers_df.insert(13, 'Floater FG%', two_pointers_df['Floaters made']/two_pointers_df['Floaters attempted'])
two_pointers_df.insert(16, 'Hook Shot FG%', two_pointers_df['Hook Shots made']/two_pointers_df['Hook Shots attempted'])
two_pointers_df.insert(19, 'Jumpshot FG%', two_pointers_df['Jumpshots made']/two_pointers_df['Jumpshots attempted'])
two_pointers_df.insert(22, 'Pullup FG%', two_pointers_df['Pullups made']/two_pointers_df['Pullups attempted'])
two_pointers_df.insert(25, 'Turnaround FG%', two_pointers_df['Turnarounds made']/two_pointers_df['Turnarounds attempted'])
two_pointers_df.insert(28, 'Step Back FG%', two_pointers_df['Step Backs made']/two_pointers_df['Step Backs attempted'])
two_pointers_df.insert(30, 'Jumpshots assisted%', two_pointers_df['Jumpshots assisted on']/two_pointers_df['2PM'])
two_pointers_df['zone 1 FG%'] = two_pointers_df['1 made']/two_pointers_df['1 attempted']
two_pointers_df['zone 2-1 FG%'] = two_pointers_df['2-1 made']/two_pointers_df['2-1 attempted']
two_pointers_df['zone 2-2 FG%'] = two_pointers_df['2-2 made']/two_pointers_df['2-2 attempted']
two_pointers_df['zone 2-3 FG%'] = two_pointers_df['2-3 made']/two_pointers_df['2-3 attempted']
two_pointers_df['zone 3-1 FG%'] = two_pointers_df['3-1 made']/two_pointers_df['3-1 attempted']
two_pointers_df['zone 3-2 FG%'] = two_pointers_df['3-2 made']/two_pointers_df['3-2 attempted']
two_pointers_df['zone 3-3 FG%'] = two_pointers_df['3-3 made']/two_pointers_df['3-3 attempted']
two_pointers_df['zone 4-1 FG%'] = two_pointers_df['4-1 made']/two_pointers_df['4-1 attempted']
two_pointers_df['zone 4-2 FG%'] = two_pointers_df['4-2 made']/two_pointers_df['4-2 attempted']
two_pointers_df['zone 4-3 FG%'] = two_pointers_df['4-3 made']/two_pointers_df['4-3 attempted']

In [52]:
two_pointers_df['2PM'] = two_pointers_df['2PM']/two_pointers_df.played
two_pointers_df['2PA'] = two_pointers_df['2PA']/two_pointers_df.played
two_pointers_df['Dunks made'] = two_pointers_df['Dunks made']/two_pointers_df.played
two_pointers_df['Dunks attempted'] = two_pointers_df['Dunks attempted']/two_pointers_df.played
two_pointers_df['Layups made'] = two_pointers_df['Layups made']/two_pointers_df.played
two_pointers_df['Layups attempted'] = two_pointers_df['Layups attempted']/two_pointers_df.played
two_pointers_df['Floaters made'] = two_pointers_df['Floaters made']/two_pointers_df.played
two_pointers_df['Floaters attempted'] = two_pointers_df['Floaters attempted']/two_pointers_df.played
two_pointers_df['Hook Shots made'] = two_pointers_df['Hook Shots made']/two_pointers_df.played
two_pointers_df['Hook Shots attempted'] = two_pointers_df['Hook Shots attempted']/two_pointers_df.played
two_pointers_df['Jumpshots made'] = two_pointers_df['Jumpshots made']/two_pointers_df.played
two_pointers_df['Jumpshots attempted'] = two_pointers_df['Jumpshots attempted']/two_pointers_df.played
two_pointers_df['Pullups made'] = two_pointers_df['Pullups made']/two_pointers_df.played
two_pointers_df['Pullups attempted'] = two_pointers_df['Pullups attempted']/two_pointers_df.played
two_pointers_df['Turnarounds made'] = two_pointers_df['Turnarounds made']/two_pointers_df.played
two_pointers_df['Turnarounds attempted'] = two_pointers_df['Turnarounds attempted']/two_pointers_df.played
two_pointers_df['Step Backs made'] = two_pointers_df['Step Backs made']/two_pointers_df.played
two_pointers_df['Step Backs attempted'] = two_pointers_df['Step Backs attempted']/two_pointers_df.played
two_pointers_df['1 made'] = two_pointers_df['1 made']/two_pointers_df.played
two_pointers_df['1 attempted'] = two_pointers_df['1 attempted']/two_pointers_df.played
two_pointers_df['2-1 made'] = two_pointers_df['2-1 made']/two_pointers_df.played
two_pointers_df['2-1 attempted'] = two_pointers_df['2-1 attempted']/two_pointers_df.played
two_pointers_df['2-2 made'] = two_pointers_df['2-2 made']/two_pointers_df.played
two_pointers_df['2-2 attempted'] = two_pointers_df['2-2 attempted']/two_pointers_df.played
two_pointers_df['2-3 made'] = two_pointers_df['2-3 made']/two_pointers_df.played
two_pointers_df['2-3 attempted'] = two_pointers_df['2-3 attempted']/two_pointers_df.played
two_pointers_df['3-1 made'] = two_pointers_df['3-1 made']/two_pointers_df.played
two_pointers_df['3-1 attempted'] = two_pointers_df['3-1 attempted']/two_pointers_df.played
two_pointers_df['3-2 made'] = two_pointers_df['3-2 made']/two_pointers_df.played
two_pointers_df['3-2 attempted'] = two_pointers_df['3-2 attempted']/two_pointers_df.played
two_pointers_df['3-3 made'] = two_pointers_df['3-3 made']/two_pointers_df.played
two_pointers_df['3-3 attempted'] = two_pointers_df['3-3 attempted']/two_pointers_df.played
two_pointers_df['4-1 made'] = two_pointers_df['4-1 made']/two_pointers_df.played
two_pointers_df['4-1 attempted'] = two_pointers_df['4-1 attempted']/two_pointers_df.played
two_pointers_df['4-2 made'] = two_pointers_df['4-2 made']/two_pointers_df.played
two_pointers_df['4-2 attempted'] = two_pointers_df['4-2 attempted']/two_pointers_df.played
two_pointers_df['4-3 made'] = two_pointers_df['4-3 made']/two_pointers_df.played
two_pointers_df['4-3 attempted'] = two_pointers_df['4-3 attempted']/two_pointers_df.played

In [53]:
two_pointers_df = two_pointers_df.add_prefix("Last_season ")
two_pointers_df = two_pointers_df.rename(columns={'Last_season player_id':'player_id'})

Feature creation for 3 pointers

In [54]:
#Filter on 3 point shots
Threes_df = shots_df[shots_df.Type=='Three']
three_pointers_df = Threes_df.groupby(['player_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'3PM', 'count':'3PA'})

In [55]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
three_pointers_df = pd.merge(player_team[['player_id','played']], three_pointers_df, how= 'left', on=['player_id']).fillna(0)

In [56]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Jumpshot', 'Pullup', 'Step Back']:
    temp = Threes_df[Threes_df[stat]==True]
    temp_df = temp.groupby(['player_id']).Result.agg(
        ['sum','count']).reset_index().rename(columns={'sum': '3P '+stat+'s made', 'count': '3P '+stat+'s attempted'})
    three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)

In [57]:
#Calculated percentage of made 3p jumpshots that were assisted
temp_df = Threes_df[(Threes_df['Jumpshot']==True)&(Threes_df['Result']==1)].groupby(
    ['player_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': '3P Jumpshots assisted on'})

In [58]:
three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)

In [59]:
#3P% per zone for each player
Zones3 = Threes_df.groupby(['player_id','Zone']).Result.agg(['sum','count']).reset_index()

In [60]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones3.pivot(index=['player_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])##remove leading space

In [61]:
three_pointers_df = pd.merge(three_pointers_df, pivot_zones, how= 'left', on=['player_id']).fillna(0)

In [62]:
#For each dataframe and each shot/zone type compute the respective percentages
three_pointers_df.insert(4, '3P%', three_pointers_df['3PM']/three_pointers_df['3PA'])
three_pointers_df.insert(7, 'Jumpshots 3P%', three_pointers_df['3P Jumpshots made']/three_pointers_df['3P Jumpshots attempted'])
three_pointers_df.insert(10, 'Pullups 3P%', three_pointers_df['3P Pullups made']/three_pointers_df['3P Pullups attempted'])
three_pointers_df.insert(13, 'Step Backs 3P%', three_pointers_df['3P Step Backs made']/three_pointers_df['3P Step Backs attempted'])
three_pointers_df.insert(16, '3P Jumpshots assisted%', three_pointers_df['3P Jumpshots assisted on']/three_pointers_df['3PM'])
three_pointers_df['zone 5-1 FG%'] = three_pointers_df['5-1 made']/three_pointers_df['5-1 attempted']
three_pointers_df['zone 5-2 FG%'] = three_pointers_df['5-2 made']/three_pointers_df['5-2 attempted']
three_pointers_df['zone 6-1 FG%'] = three_pointers_df['6-1 made']/three_pointers_df['6-1 attempted']
three_pointers_df['zone 6-2 FG%'] = three_pointers_df['6-2 made']/three_pointers_df['6-2 attempted']
three_pointers_df['zone 6-3 FG%'] = three_pointers_df['6-3 made']/three_pointers_df['6-3 attempted']
three_pointers_df['zone 7-1 FG%'] = three_pointers_df['7-1 made']/three_pointers_df['7-1 attempted']
three_pointers_df['zone 7-2 FG%'] = three_pointers_df['7-2 made']/three_pointers_df['7-2 attempted']
three_pointers_df['zone 7-3 FG%'] = three_pointers_df['7-3 made']/three_pointers_df['7-3 attempted']

In [63]:
three_pointers_df['3PM'] = three_pointers_df['3PM']/three_pointers_df.played
three_pointers_df['3PA'] = three_pointers_df['3PA']/three_pointers_df.played
three_pointers_df['3P Jumpshots made'] = three_pointers_df['3P Jumpshots made']/three_pointers_df.played
three_pointers_df['3P Jumpshots attempted'] = three_pointers_df['3P Jumpshots attempted']/three_pointers_df.played
three_pointers_df['3P Pullups made'] = three_pointers_df['3P Pullups made']/three_pointers_df.played
three_pointers_df['3P Pullups attempted'] = three_pointers_df['3P Pullups attempted']/three_pointers_df.played
three_pointers_df['3P Step Backs made'] = three_pointers_df['3P Step Backs made']/three_pointers_df.played
three_pointers_df['3P Step Backs attempted'] = three_pointers_df['3P Step Backs attempted']/three_pointers_df.played
three_pointers_df['5-1 made'] = three_pointers_df['5-1 made']/three_pointers_df.played
three_pointers_df['5-1 attempted'] = three_pointers_df['5-1 attempted']/three_pointers_df.played
three_pointers_df['5-2 made'] = three_pointers_df['5-2 made']/three_pointers_df.played
three_pointers_df['5-2 attempted'] = three_pointers_df['5-2 attempted']/three_pointers_df.played
three_pointers_df['6-1 made'] = three_pointers_df['6-1 made']/three_pointers_df.played
three_pointers_df['6-1 attempted'] = three_pointers_df['6-1 attempted']/three_pointers_df.played
three_pointers_df['6-2 made'] = three_pointers_df['6-2 made']/three_pointers_df.played
three_pointers_df['6-2 attempted'] = three_pointers_df['6-2 attempted']/three_pointers_df.played
three_pointers_df['6-3 made'] = three_pointers_df['6-3 made']/three_pointers_df.played
three_pointers_df['6-3 attempted'] = three_pointers_df['6-3 attempted']/three_pointers_df.played
three_pointers_df['7-1 made'] = three_pointers_df['7-1 made']/three_pointers_df.played
three_pointers_df['7-1 attempted'] = three_pointers_df['7-1 attempted']/three_pointers_df.played
three_pointers_df['7-2 made'] = three_pointers_df['7-2 made']/three_pointers_df.played
three_pointers_df['7-2 attempted'] = three_pointers_df['7-2 attempted']/three_pointers_df.played
three_pointers_df['7-3 made'] = three_pointers_df['7-3 made']/three_pointers_df.played
three_pointers_df['7-3 attempted'] = three_pointers_df['7-3 attempted']/three_pointers_df.played


In [64]:
three_pointers_df = three_pointers_df.add_prefix("Last_season ")
three_pointers_df = three_pointers_df.rename(columns={'Last_season player_id':'player_id'})

### Current Season

In [65]:
#Import neccessary pickle files
games = dataframes_cs["games_df"].copy()
shots = dataframes_cs['shots_df'].copy()
player = dataframes_cs['players_df'].copy()
segment = dataframes_cs['segments_df'].copy()
Last_season_2P_features_df = two_pointers_df.copy()
Last_season_3P_features_df = three_pointers_df.copy()

In [66]:
segment = segment[['game_id','players_team1','players_team2']]
segment["all_players"] = segment["players_team1"]+segment["players_team2"]
segments_df_long = segment.explode(["all_players"])
segments_df_long = segments_df_long.drop(columns=['players_team1','players_team2'])
segments_df_long = segments_df_long.drop_duplicates()


In [67]:
segments_df_long['played']=1

Create a dataframe with all possible combinations of team_id, game_id, player_id

In [68]:
#First we will melt the games dataframe two combine the team 1 and and team 2 columns into one
games_df = pd.melt(games, id_vars=['game_id','season','league','date','has_locations','has_segments','hometeam'], 
                   value_vars=['team1', 'team2'], value_name='team_id').drop(columns=['variable'])
#Next, we will sort the dataframe from newest to oldest date
games_df.date = games_df.date.apply(pd.to_datetime)
games_df = games_df.sort_values(['date'], ascending=False)

In [69]:
#create a new column to track the number of games each team has played since the current one
games_df['games played since'] = games_df.groupby(games_df.team_id).cumcount()

In [70]:
#merge the players with the shots dataframe to add the team_id column 
#that is neccessary for merging with the games dataframe
player_team = pd.merge(player.rename(columns={'seasonid':'season_id'}), shots[['team_id','season_id']].drop_duplicates(),
                        how='left', on='season_id')

In [71]:
#Finally create the dataframe with all the neccessary columns 
team_player_game_df = pd.merge(player_team[['team_id','player']], games_df[['team_id','game_id','games played since']], 
                               on='team_id').rename(columns={'player':'player_id'})

# team_player_game_df.to_pickle("team_player_game_df.pickle") 

In [72]:
#Find the maximum number of games for one team (will be used later)
most_team_games = team_player_game_df['games played since'].max()+1

In [73]:
team_player_game_df = pd.merge(team_player_game_df, segments_df_long.rename(columns={'all_players':'player_id'}), on=['game_id','player_id'], how='left')
team_player_game_df.played = team_player_game_df.played.fillna(0).astype(int)


In [74]:
#Split the Stat column into 2s/3s and Makes/Misses
shots_df = shots.copy()
shots_df.insert(13, 'Type', shots_df.Stat.str[:-4])
shots_df.insert(14, 'Result', shots_df.Stat.str[-4:])
shots_df.Result = np.where(shots_df.Result=='Make', 1, 0)

In [75]:
#Column D1 is a binary True/False variable that keeps track of whether or not that takes the shots plays for a D1 team
player['D1'] = True
shots_df = pd.merge(shots_df, player[['player', 'D1']].rename(columns={'player':'player_id'}), on='player_id', how='left')
shots_df['D1'] = shots_df['D1'].fillna(False)

Feature creation for 2 pointers

In [76]:
#Filter on 2 point shots
Twos_df = shots_df[shots_df.Type=='Two']
two_pointers_df = Twos_df.groupby(['player_id','game_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'2PM', 'count':'2PA'})

In [77]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
two_pointers_df = pd.merge(team_player_game_df, two_pointers_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [78]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Dunk', 'Layup', 'Floater', 'Hook Shot', 'Jumpshot', 'Pullup', 'Turnaround', 'Step Back']:
    temp = Twos_df[Twos_df[stat]==True]
    temp_df = temp.groupby(['player_id','game_id']).Result.agg(['sum','count']).reset_index().rename(
        columns={'sum': stat+'s made', 'count': stat+'s attempted'})
    two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [79]:
#Calculated percentage of made 2p jumpshots that were assisted
temp_df = Twos_df[(Twos_df['Jumpshot']==True)&(Twos_df['Result']==1)].groupby(
    ['player_id','game_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': 'Jumpshots assisted on'})

In [80]:
two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [81]:
#2P% per zone for each player
Zones2 = Twos_df.groupby(['player_id','game_id','Zone']).Result.agg(['sum','count']).reset_index()

In [82]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones2.pivot(index=['player_id','game_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])#remove leading space

In [83]:
two_pointers_df = pd.merge(two_pointers_df, pivot_zones, how= 'left', on=['player_id','game_id']).fillna(0)

In [84]:
#Create a new dataframe for the last 1, 3, 5, 10 games, plus the season total
#Use the rolling function to compute the last x games sums, NOT INCLUDING the game in each row (closed='left')
Last_game_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=1).sum()
Last_3_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=3).sum()
Last_5_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=5).sum()
Last_10_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=10).sum()
Season_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(
        closed='left',window=most_team_games, min_periods=1).sum()

In [85]:
#For each dataframe and each shot/zone type compute the respective percentages
for df in [Last_game_2P, Last_3_games_2P, Last_5_games_2P, Last_10_games_2P, Season_2P]:
    df.insert(5, '2P%', df['2PM']/df['2PA'])
    df.insert(8, 'Dunk FG%', df['Dunks made']/df['Dunks attempted'])
    df.insert(11, 'Layup FG%', df['Layups made']/df['Layups attempted'])
    df.insert(14, 'Floater FG%', df['Floaters made']/df['Floaters attempted'])
    df.insert(17, 'Hook Shot FG%', df['Hook Shots made']/df['Hook Shots attempted'])
    df.insert(20, 'Jumpshot FG%', df['Jumpshots made']/df['Jumpshots attempted'])
    df.insert(23, 'Pullup FG%', df['Pullups made']/df['Pullups attempted'])
    df.insert(26, 'Turnaround FG%', df['Turnarounds made']/df['Turnarounds attempted'])
    df.insert(29, 'Step Back FG%', df['Step Backs made']/df['Step Backs attempted'])
    df.insert(31, 'Jumpshots assisted%', df['Jumpshots assisted on']/df['2PM'])
    df['zone 1 FG%'] = df['1 made']/df['1 attempted']
    df['zone 2-1 FG%'] = df['2-1 made']/df['2-1 attempted']
    df['zone 2-2 FG%'] = df['2-2 made']/df['2-2 attempted']
    df['zone 2-3 FG%'] = df['2-3 made']/df['2-3 attempted']
    df['zone 3-1 FG%'] = df['3-1 made']/df['3-1 attempted']
    df['zone 3-2 FG%'] = df['3-2 made']/df['3-2 attempted']
    df['zone 3-3 FG%'] = df['3-3 made']/df['3-3 attempted']
    df['zone 4-1 FG%'] = df['4-1 made']/df['4-1 attempted']
    df['zone 4-2 FG%'] = df['4-2 made']/df['4-2 attempted']
    df['zone 4-3 FG%'] = df['4-3 made']/df['4-3 attempted']

In [86]:
#For each dataframe and each shot/zone type normalize the respective makes and attempts
for df in [Last_game_2P, Last_3_games_2P, Last_5_games_2P, Last_10_games_2P, Season_2P]:
    df['2PM'] = np.where(df.played.isna()==True, df['2PM'], df['2PM']/df.played)
    df['2PA'] = np.where(df.played.isna()==True, df['2PA'], df['2PA']/df.played)
    df['Dunks made'] = np.where(df.played.isna()==True, df['Dunks made'], df['Dunks made']/df.played)
    df['Dunks attempted'] = np.where(df.played.isna()==True, df['Dunks attempted'], df['Dunks attempted']/df.played)
    df['Layups made'] = np.where(df.played.isna()==True, df['Layups made'], df['Layups made']/df.played)
    df['Layups attempted'] = np.where(df.played.isna()==True, df['Layups attempted'], df['Layups attempted']/df.played)
    df['Floaters made'] = np.where(df.played.isna()==True, df['Floaters made'], df['Floaters made']/df.played)
    df['Floaters attempted'] = np.where(df.played.isna()==True, df['Floaters attempted'], df['Floaters attempted']/df.played)
    df['Hook Shots made'] = np.where(df.played.isna()==True, df['Hook Shots made'], df['Hook Shots made']/df.played)
    df['Hook Shots attempted'] = np.where(df.played.isna()==True, df['Hook Shots attempted'], df['Hook Shots attempted']/df.played)
    df['Jumpshots made'] = np.where(df.played.isna()==True, df['Jumpshots made'], df['Jumpshots made']/df.played)
    df['Jumpshots attempted'] = np.where(df.played.isna()==True, df['Jumpshots attempted'], df['Jumpshots attempted']/df.played)
    df['Pullups made'] = np.where(df.played.isna()==True, df['Pullups made'], df['Pullups made']/df.played)
    df['Pullups attempted'] = np.where(df.played.isna()==True, df['Pullups attempted'], df['Pullups attempted']/df.played)
    df['Turnarounds made'] = np.where(df.played.isna()==True, df['Turnarounds made'], df['Turnarounds made']/df.played)
    df['Turnarounds attempted'] = np.where(df.played.isna()==True, df['Turnarounds attempted'], df['Turnarounds attempted']/df.played)
    df['Step Backs made'] = np.where(df.played.isna()==True, df['Step Backs made'], df['Step Backs made']/df.played)
    df['Step Backs attempted'] = np.where(df.played.isna()==True, df['Step Backs attempted'], df['Step Backs attempted']/df.played)
    df['1 made'] = np.where(df.played.isna()==True, df['1 made'], df['1 made']/df.played)
    df['1 attempted'] = np.where(df.played.isna()==True, df['1 attempted'], df['1 attempted']/df.played)
    df['2-1 made'] = np.where(df.played.isna()==True, df['2-1 made'], df['2-1 made']/df.played)
    df['2-1 attempted'] = np.where(df.played.isna()==True, df['2-1 attempted'], df['2-1 attempted']/df.played)
    df['2-2 made'] = np.where(df.played.isna()==True, df['2-2 made'], df['2-2 made']/df.played)
    df['2-2 attempted'] = np.where(df.played.isna()==True, df['2-2 attempted'], df['2-2 attempted']/df.played)
    df['2-3 made'] = np.where(df.played.isna()==True, df['2-3 made'], df['2-3 made']/df.played)
    df['2-3 attempted'] = np.where(df.played.isna()==True, df['2-3 attempted'], df['2-3 attempted']/df.played)
    df['3-1 made'] = np.where(df.played.isna()==True, df['3-1 made'], df['3-1 made']/df.played)
    df['3-1 attempted'] = np.where(df.played.isna()==True, df['3-1 attempted'], df['3-1 attempted']/df.played)
    df['3-2 made'] = np.where(df.played.isna()==True, df['3-2 made'], df['3-2 made']/df.played)
    df['3-2 attempted'] = np.where(df.played.isna()==True, df['3-2 attempted'], df['3-2 attempted']/df.played)
    df['3-3 made'] = np.where(df.played.isna()==True, df['3-3 made'], df['3-3 made']/df.played)
    df['3-3 attempted'] = np.where(df.played.isna()==True, df['3-3 attempted'], df['3-3 attempted']/df.played)
    df['4-1 made'] = np.where(df.played.isna()==True, df['4-1 made'], df['4-1 made']/df.played)
    df['4-1 attempted'] = np.where(df.played.isna()==True, df['4-1 attempted'], df['4-1 attempted']/df.played)
    df['4-2 made'] = np.where(df.played.isna()==True, df['4-2 made'], df['4-2 made']/df.played)
    df['4-2 attempted'] = np.where(df.played.isna()==True, df['4-2 attempted'], df['4-2 attempted']/df.played)
    df['4-3 made'] = np.where(df.played.isna()==True, df['4-3 made'], df['4-3 made']/df.played)
    df['4-3 attempted'] = np.where(df.played.isna()==True, df['4-3 attempted'], df['4-3 attempted']/df.played)

In [87]:
#Rename the columns before merging all the dataframes together
Last_game_2P.columns = Last_game_2P.columns.map(lambda x: f'L1 {x}')
Last_3_games_2P.columns = Last_3_games_2P.columns.map(lambda x: f'L3 {x}')
Last_5_games_2P.columns = Last_5_games_2P.columns.map(lambda x: f'L5 {x}')
Last_10_games_2P.columns = Last_10_games_2P.columns.map(lambda x: f'L10 {x}')
Season_2P.columns = Season_2P.columns.map(lambda x: f'TOT {x}')
Last_game_2P = Last_game_2P.rename(columns={'L1 team_id':'team_id','L1 player_id':'player_id'})
Last_3_games_2P = Last_3_games_2P.rename(columns={'L3 team_id':'team_id','L3 player_id':'player_id'})
Last_5_games_2P = Last_5_games_2P.rename(columns={'L5 team_id':'team_id','L5 player_id':'player_id'})
Last_10_games_2P = Last_10_games_2P.rename(columns={'L10 team_id':'team_id','L10 player_id':'player_id'})
Season_2P = Season_2P.rename(columns={'TOT team_id':'team_id','TOT player_id':'player_id'})

In [88]:
#Concatenate all the dataframes into one
feature_df_2P = pd.concat([team_player_game_df, Last_game_2P.iloc[:,2:], 
                           Last_3_games_2P.iloc[:,2:], Last_5_games_2P.iloc[:,2:], 
                           Last_10_games_2P.iloc[:,2:], Season_2P.iloc[:,2:]], axis=1)
# feature_df_2P.to_pickle("2P_features_df.pickle")#write the data to a pickle file
# feature_df_2P
#Now that current season's features are created, we will merge them with last season's features


In [89]:
reduced_2P_df = pd.merge(Twos_df, feature_df_2P, how='left', on=['team_id','player_id','game_id'])
reduced_2P_df = pd.merge(reduced_2P_df, Last_season_2P_features_df, how='left', on=['player_id'])

In [90]:
NAs2 = reduced_2P_df.isna().sum()/reduced_2P_df.shape[0]

In [91]:
NAs2[42::60]

L1 2PM             0.103271
L3 2PM             0.141591
L5 2PM             0.200342
L10 2PM            0.352652
TOT 2PM            0.074270
Last_season 2PM    0.257877
dtype: float64

In [92]:
reduced_2P_df.insert(41, 'Last_season Zone perc%', np.nan)
reduced_2P_df.insert(41, 'Last_season Zone attempted', 0)
reduced_2P_df.insert(41, 'Last_season Zone made', 0)
reduced_2P_df.insert(41, 'TOT Zone perc%', np.nan)
reduced_2P_df.insert(41, 'TOT Zone attempted', 0)
reduced_2P_df.insert(41, 'TOT Zone made', 0)
reduced_2P_df.insert(41, 'L10 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L10 Zone attempted', 0)
reduced_2P_df.insert(41, 'L10 Zone made', 0)
reduced_2P_df.insert(41, 'L5 Zone attempted', 0)
reduced_2P_df.insert(41, 'L5 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L5 Zone made', 0)
reduced_2P_df.insert(41, 'L3 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L3 Zone attempted', 0)
reduced_2P_df.insert(41, 'L3 Zone made', 0)
reduced_2P_df.insert(41, 'L1 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L1 Zone attempted', 0)
reduced_2P_df.insert(41, 'L1 Zone made', 0)

In [93]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for zone in [' 1',' 2-1',' 2-2',' 2-3',' 3-1',' 3-2',' 3-3',' 4-1',' 4-2',' 4-3']:
        reduced_2P_df[f"{time} Zone made"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time}{zone} made"], reduced_2P_df[f"{time} Zone made"])
        cols_to_drop.append(f"{time}{zone} made")
        reduced_2P_df[f"{time} Zone attempted"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time}{zone} attempted"], reduced_2P_df[f"{time} Zone attempted"])
        cols_to_drop.append(f"{time}{zone} attempted")
        reduced_2P_df[f"{time} Zone perc%"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time} zone{zone} FG%"], reduced_2P_df[f"{time} Zone perc%"])
        cols_to_drop.append(f"{time} zone{zone} FG%")
            

In [94]:
reduced_2P_df = reduced_2P_df.drop(columns=cols_to_drop)

In [95]:
reduced_2P_df.insert(56, 'Shot type', 'Other')
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Jumpshot==True, 'Jumpshot', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Pullup==True, 'Pullup', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df['Step Back']==True, 'Step Back', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Floater==True, 'Floater', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Layup==True, 'Layup', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df['Hook Shot']==True, 'Hook Shot', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Dunk==True, 'Dunk', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Turnaround==True, 'Turnaround', reduced_2P_df['Shot type'])


In [96]:
reduced_2P_df['Shot type'].value_counts()

Shot type
Layup         257018
Jumpshot      139472
Pullup         21943
Turnaround     10990
Floater         9224
Hook Shot       5936
Step Back       3011
Other             58
Dunk              53
Name: count, dtype: int64

In [97]:
reduced_2P_df.insert(57, 'Last_season Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'Last_season Shot type attempted', 0)
reduced_2P_df.insert(57, 'Last_season Shot type made', 0)
reduced_2P_df.insert(57, 'TOT Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'TOT Shot type attempted', 0)
reduced_2P_df.insert(57, 'TOT Shot type made', 0)
reduced_2P_df.insert(57, 'L10 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L10 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L10 Shot type made', 0)
reduced_2P_df.insert(57, 'L5 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L5 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L5 Shot type made', 0)
reduced_2P_df.insert(57, 'L3 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L3 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L3 Shot type made', 0)
reduced_2P_df.insert(57, 'L1 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L1 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L1 Shot type made', 0)

In [98]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for shot_type in ['Dunk','Hook Shot','Layup','Floater','Jumpshot','Pullup','Turnaround','Step Back']:
        reduced_2P_df[f"{time} Shot type made"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type}s made"], reduced_2P_df[f"{time} Shot type made"])
        cols_to_drop.append(f"{time} {shot_type}s made")
        reduced_2P_df[f"{time} Shot type attempted"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type}s attempted"], reduced_2P_df[f"{time} Shot type attempted"])
        cols_to_drop.append(f"{time} {shot_type}s attempted")
        reduced_2P_df[f"{time} Shot type perc%"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type} FG%"], reduced_2P_df[f"{time} Shot type perc%"])
        cols_to_drop.append(f"{time} {shot_type} FG%")


In [99]:
reduced_2P_df = reduced_2P_df.drop(columns=cols_to_drop)

In [100]:
condensed_per_game_2P_features = list(range(-73,0))
condensed_per_game_2P_features.insert(0,38)
condensed_per_game_2P_features.insert(0,4)

In [101]:
Condensed_2P_features_df = reduced_2P_df.iloc[:,condensed_per_game_2P_features]
# .to_pickle("Condensed_2P_features_df.pickle")
# reduced_2P_df.iloc[:,condensed_per_game_2P_features]

Feature creation for 3 pointers

In [102]:
#Filter on 3 point shots
Threes_df = shots_df[shots_df.Type=='Three']
three_pointers_df = Threes_df.groupby(['player_id','game_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'3PM', 'count':'3PA'})

In [103]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
three_pointers_df = pd.merge(team_player_game_df, three_pointers_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [104]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Jumpshot', 'Pullup', 'Step Back']:
    temp = Threes_df[Threes_df[stat]==True]
    temp_df = temp.groupby(['player_id','game_id']).Result.agg(
        ['sum','count']).reset_index().rename(columns={'sum': '3P '+stat+'s made', 'count': '3P '+stat+'s attempted'})
    three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [105]:
#Calculated percentage of made 3p jumpshots that were assisted
temp_df = Threes_df[(Threes_df['Jumpshot']==True)&(Threes_df['Result']==1)].groupby(
    ['player_id','game_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': '3P Jumpshots assisted on'})

In [106]:
three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)

In [107]:
#3P% per zone for each player
Zones3 = Threes_df.groupby(['player_id','game_id','Zone']).Result.agg(['sum','count']).reset_index()

In [108]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones3.pivot(index=['player_id','game_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])##remove leading space

In [109]:
three_pointers_df = pd.merge(three_pointers_df, pivot_zones, how= 'left', on=['player_id','game_id']).fillna(0)

In [110]:
#Create a new dataframe for the last 1, 3, 5, 10 games, plus the season total
#Use the rolling function to compute the last x games sums, NOT INCLUDING the game in each row (closed='left')
Last_game_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=1).sum()
Last_3_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=3).sum()
Last_5_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=5).sum()
Last_10_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=10).sum()
Season_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(
        closed='left',window=most_team_games, min_periods=1).sum()

In [111]:
#For each dataframe and each shot/zone type compute the respective percentages
for df in [Last_game_3P, Last_3_games_3P, Last_5_games_3P, Last_10_games_3P, Season_3P]:
    df.insert(5, '3P%', df['3PM']/df['3PA'])
    df.insert(8, 'Jumpshots 3P%', df['3P Jumpshots made']/df['3P Jumpshots attempted'])
    df.insert(11, 'Pullups 3P%', df['3P Pullups made']/df['3P Pullups attempted'])
    df.insert(14, 'Step Backs 3P%', df['3P Step Backs made']/df['3P Step Backs attempted'])
    df.insert(16, '3P Jumpshots assisted%', df['3P Jumpshots assisted on']/df['3PM'])
    df['zone 5-1 FG%'] = df['5-1 made']/df['5-1 attempted']
    df['zone 5-2 FG%'] = df['5-2 made']/df['5-2 attempted']
    df['zone 6-1 FG%'] = df['6-1 made']/df['6-1 attempted']
    df['zone 6-2 FG%'] = df['6-2 made']/df['6-2 attempted']
    df['zone 6-3 FG%'] = df['6-3 made']/df['6-3 attempted']
    df['zone 7-1 FG%'] = df['7-1 made']/df['7-1 attempted']
    df['zone 7-2 FG%'] = df['7-2 made']/df['7-2 attempted']
    df['zone 7-3 FG%'] = df['7-3 made']/df['7-3 attempted']

In [112]:
#For each dataframe and each shot/zone type normalize the respective makes and attempts
for df in [Last_game_3P, Last_3_games_3P, Last_5_games_3P, Last_10_games_3P, Season_3P]:
    df['3PM'] = np.where(df.played.isna()==True, df['3PM'], df['3PM']/df.played)
    df['3PA'] = np.where(df.played.isna()==True, df['3PA'], df['3PA']/df.played)
    df['3P Jumpshots made'] = np.where(df.played.isna()==True, df['3P Jumpshots made'], df['3P Jumpshots made']/df.played)
    df['3P Jumpshots attempted'] = np.where(df.played.isna()==True, df['3P Jumpshots attempted'], df['3P Jumpshots attempted']/df.played)
    df['3P Pullups made'] = np.where(df.played.isna()==True, df['3P Pullups made'], df['3P Pullups made']/df.played)
    df['3P Pullups attempted'] = np.where(df.played.isna()==True, df['3P Pullups attempted'], df['3P Pullups attempted']/df.played)
    df['3P Step Backs made'] = np.where(df.played.isna()==True, df['3P Step Backs made'], df['3P Step Backs made']/df.played)
    df['3P Step Backs attempted'] = np.where(df.played.isna()==True, df['3P Step Backs attempted'], df['3P Step Backs attempted']/df.played)
    df['5-1 made'] = np.where(df.played.isna()==True, df['5-1 made'], df['5-1 made']/df.played)
    df['5-1 attempted'] = np.where(df.played.isna()==True, df['5-1 attempted'], df['5-1 attempted']/df.played)
    df['5-2 made'] = np.where(df.played.isna()==True, df['5-2 made'], df['5-2 made']/df.played)
    df['5-2 attempted'] = np.where(df.played.isna()==True, df['5-2 attempted'], df['5-2 attempted']/df.played)
    df['6-1 made'] = np.where(df.played.isna()==True, df['6-1 made'], df['6-1 made']/df.played)
    df['6-1 attempted'] = np.where(df.played.isna()==True, df['6-1 attempted'], df['6-1 attempted']/df.played)
    df['6-2 made'] = np.where(df.played.isna()==True, df['6-2 made'], df['6-2 made']/df.played)
    df['6-2 attempted'] = np.where(df.played.isna()==True, df['6-2 attempted'], df['6-2 attempted']/df.played)
    df['6-3 made'] = np.where(df.played.isna()==True, df['6-3 made'], df['6-3 made']/df.played)
    df['6-3 attempted'] = np.where(df.played.isna()==True, df['6-3 attempted'], df['6-3 attempted']/df.played)
    df['7-1 made'] = np.where(df.played.isna()==True, df['7-1 made'], df['7-1 made']/df.played)
    df['7-1 attempted'] = np.where(df.played.isna()==True, df['7-1 attempted'], df['7-1 attempted']/df.played)
    df['7-2 made'] = np.where(df.played.isna()==True, df['7-2 made'], df['7-2 made']/df.played)
    df['7-2 attempted'] = np.where(df.played.isna()==True, df['7-2 attempted'], df['7-2 attempted']/df.played)
    df['7-3 made'] = np.where(df.played.isna()==True, df['7-3 made'], df['7-3 made']/df.played)
    df['7-3 attempted'] = np.where(df.played.isna()==True, df['7-3 attempted'], df['7-3 attempted']/df.played)

In [113]:
#Rename the columns before merging all the dataframes together
Last_game_3P.columns = Last_game_3P.columns.map(lambda x: f'L1 {x}')
Last_3_games_3P.columns = Last_3_games_3P.columns.map(lambda x: f'L3 {x}')
Last_5_games_3P.columns = Last_5_games_3P.columns.map(lambda x: f'L5 {x}')
Last_10_games_3P.columns = Last_10_games_3P.columns.map(lambda x: f'L10 {x}')
Season_3P.columns = Season_3P.columns.map(lambda x: f'TOT {x}')
Last_game_3P = Last_game_3P.rename(columns={'L1 team_id':'team_id','L1 player_id':'player_id'})
Last_3_games_3P = Last_3_games_3P.rename(columns={'L3 team_id':'team_id','L3 player_id':'player_id'})
Last_5_games_3P = Last_5_games_3P.rename(columns={'L5 team_id':'team_id','L5 player_id':'player_id'})
Last_10_games_3P = Last_10_games_3P.rename(columns={'L10 team_id':'team_id','L10 player_id':'player_id'})
Season_3P = Season_3P.rename(columns={'TOT team_id':'team_id','TOT player_id':'player_id'})

In [114]:
#Concatenate all the dataframes into one
feature_df_3P = pd.concat([team_player_game_df, Last_game_3P.iloc[:,2:], 
                           Last_3_games_3P.iloc[:,2:], Last_5_games_3P.iloc[:,2:], 
                           Last_10_games_3P.iloc[:,2:], Season_3P.iloc[:,2:]], axis=1)
# feature_df_3P.to_pickle("3P_features_df.pickle")#write the data to a pickle file
#Now that current season's features are created, we will merge them with last season's features


In [115]:
reduced_3P_df = pd.merge(Threes_df, feature_df_3P, how='left', on=['team_id','player_id','game_id'])
reduced_3P_df = pd.merge(reduced_3P_df, Last_season_3P_features_df, how='left', on=['player_id'])

In [116]:
NAs3 = reduced_3P_df.isna().sum()/reduced_3P_df.shape[0]

In [117]:
reduced_3P_df.insert(41, 'Last_season Zone perc%', np.nan)
reduced_3P_df.insert(41, 'Last_season Zone attempted', 0)
reduced_3P_df.insert(41, 'Last_season Zone made', 0)
reduced_3P_df.insert(41, 'TOT Zone perc%', np.nan)
reduced_3P_df.insert(41, 'TOT Zone attempted', 0)
reduced_3P_df.insert(41, 'TOT Zone made', 0)
reduced_3P_df.insert(41, 'L10 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L10 Zone attempted', 0)
reduced_3P_df.insert(41, 'L10 Zone made', 0)
reduced_3P_df.insert(41, 'L5 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L5 Zone attempted', 0)
reduced_3P_df.insert(41, 'L5 Zone made', 0)
reduced_3P_df.insert(41, 'L3 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L3 Zone attempted', 0)
reduced_3P_df.insert(41, 'L3 Zone made', 0)
reduced_3P_df.insert(41, 'L1 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L1 Zone attempted', 0)
reduced_3P_df.insert(41, 'L1 Zone made', 0)


In [118]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for zone in [' 5-1',' 5-2',' 6-1',' 6-2',' 6-3',' 7-1',' 7-2',' 7-3']:
        reduced_3P_df[f"{time} Zone made"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time}{zone} made"], reduced_3P_df[f"{time} Zone made"])
        cols_to_drop.append(f"{time}{zone} made")
        reduced_3P_df[f"{time} Zone attempted"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time}{zone} attempted"], reduced_3P_df[f"{time} Zone attempted"])
        cols_to_drop.append(f"{time}{zone} attempted")
        reduced_3P_df[f"{time} Zone perc%"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time} zone{zone} FG%"], reduced_3P_df[f"{time} Zone perc%"])
        cols_to_drop.append(f"{time} zone{zone} FG%")


In [119]:
reduced_3P_df = reduced_3P_df.drop(columns=cols_to_drop)

In [120]:
reduced_3P_df.insert(59, 'Shot type', 'Other')
reduced_3P_df['Shot type'] = np.where(reduced_3P_df.Jumpshot==True, 'Jumpshot', reduced_3P_df['Shot type'])
reduced_3P_df['Shot type'] = np.where(reduced_3P_df.Pullup==True, 'Pullup', reduced_3P_df['Shot type'])
reduced_3P_df['Shot type'] = np.where(reduced_3P_df['Step Back']==True, 'Step Back', reduced_3P_df['Shot type'])


In [121]:
reduced_3P_df.insert(60, 'Last_season Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'Last_season Shot type attempted', 0)
reduced_3P_df.insert(60, 'Last_season Shot type made', 0)
reduced_3P_df.insert(60, 'TOT Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'TOT Shot type attempted', 0)
reduced_3P_df.insert(60, 'TOT Shot type made', 0)
reduced_3P_df.insert(60, 'L10 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L10 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L10 Shot type made', 0)
reduced_3P_df.insert(60, 'L5 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L5 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L5 Shot type made', 0)
reduced_3P_df.insert(60, 'L3 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L3 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L3 Shot type made', 0)
reduced_3P_df.insert(60, 'L1 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L1 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L1 Shot type made', 0)

In [122]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for shot_type in ['Jumpshot','Pullup','Step Back']:
        reduced_3P_df[f"{time} Shot type made"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} 3P {shot_type}s made"], reduced_3P_df[f"{time} Shot type made"])
        cols_to_drop.append(f"{time} 3P {shot_type}s made")
        reduced_3P_df[f"{time} Shot type attempted"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} 3P {shot_type}s attempted"], reduced_3P_df[f"{time} Shot type attempted"])
        cols_to_drop.append(f"{time} 3P {shot_type}s attempted")
        reduced_3P_df[f"{time} Shot type perc%"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} {shot_type}s 3P%"], reduced_3P_df[f"{time} Shot type perc%"])
        cols_to_drop.append(f"{time} {shot_type}s 3P%")


In [123]:
reduced_3P_df = reduced_3P_df.drop(columns=cols_to_drop)

In [124]:
condensed_per_game_3P_features = list(range(-73,0))
condensed_per_game_3P_features.insert(0,38)
condensed_per_game_3P_features.insert(0,4)

In [125]:
Condensed_3P_features_df = reduced_3P_df.iloc[:,condensed_per_game_3P_features]
# .to_pickle("Condensed_3P_features_df.pickle")
#Condensed_3P_features_df,Condensed_2P_features_df

# Grand Merge

In [126]:
df_features_additional = features_part_2
print(df_features_additional.shape)
print(merged_df.shape)
merged_df_2 = pd.merge(merged_df, df_features_additional, on=['shot_id'], how='left')
print(merged_df_2.shape)

(664654, 14)
(664654, 59)
(664654, 72)


Splitting into 2 pointer & 3 pointer datasets

In [127]:
merged_df_two_pointers = merged_df_2[merged_df_2['Stat'].isin(['TwoMake', 'TwoMiss'])].copy()
merged_df_three_pointers = merged_df_2[merged_df_2['Stat'].isin(['ThreeMake', 'ThreeMiss'])].copy()

merged_df_two_pointers['Made'] = merged_df_two_pointers['Stat'] == 'TwoMake'
merged_df_three_pointers['Made'] = merged_df_three_pointers['Stat'] == 'ThreeMake'

Adding Nikos's features

In [128]:
feature_df_3P = Condensed_3P_features_df
feature_df_2P = Condensed_2P_features_df

print(feature_df_3P.shape)
print(feature_df_2P.shape)

(216949, 75)
(447705, 75)


In [130]:
merged_df_with_2P = pd.merge(merged_df_two_pointers, feature_df_2P, on=['shot_id'], how='left')
merged_df_with_3P = pd.merge(merged_df_three_pointers, feature_df_3P, on=['shot_id'], how='left')

print(merged_df_with_2P.shape)
print(merged_df_with_3P.shape)

(447705, 147)
(216949, 147)


In [131]:
merged_df_with_2P.to_pickle(f'data/processed_data/{league}_{season}_2P.pickle')
merged_df_with_3P.to_pickle(f'data/processed_data/{league}_{season}_3P.pickle')

### Compare with old Dataset

In [ ]:
merged_df_with_2P_old = pd.read_csv(f'data/processed_data/Women_merged_df_with_2P.csv').drop(columns="Unnamed: 0")
merged_df_with_3P_old = pd.read_csv(f'data/processed_data/Women_merged_df_with_3P.csv').drop(columns="Unnamed: 0")
 

In [ ]:
columns = merged_df_with_2P.columns.to_list()
columns

In [ ]:
a = merged_df_with_2P[columns].compare(merged_df_with_2P_old[columns])

In [ ]:
merged_df_with_2P_old.shape, merged_df_with_2P.shape

In [ ]:
a